In [20]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import NeuralNetwork as NN
import Layer
import importlib
importlib.reload(NN)
importlib.reload(Layer)

<module 'Layer' from '/Users/jorchard/Dropbox/research/peBogacz/python/Layer.py'>

In [21]:
net = NN.NeuralNetwork()

In [22]:
l = Layer.PELayer(n=5)
net.AddLayer(l)
net.AddLayer(Layer.PELayer(n=3))

In [23]:
x = torch.randn(5)
net.layers[0].Set(x)

In [24]:
net.layers[0].e

tensor([ 0.,  0.,  0.,  0.,  0.])

In [25]:
net.M

[tensor([[ 2.6786, -0.1966, -0.5937],
         [-0.3070, -2.2439,  0.1771],
         [ 0.0521, -0.4778,  0.4728],
         [-0.1906,  0.4791, -0.4368],
         [ 1.7054,  0.9731, -1.5578]])]

In [26]:
net.layers[0].v

tensor([ 1.3589, -0.3366, -0.4540,  0.9185,  0.2284])

In [27]:
len(net.layers)

2

In [28]:
torch.mv( net.W[0], net.layers[0].Output_Up() )

tensor([ 0.,  0.,  0.])

In [31]:
net.layers[1].IntegrateFromBelow( torch.mv(net.W[0], net.layers[0].Output_Up()) )

UnboundLocalError: local variable 'dvdt' referenced before assignment